In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from matplotlib import pyplot as plt
import os
import pandas as pd
from PIL import Image


In [2]:
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms

In [3]:
torch.manual_seed(42)
np.random.seed(42)

In [4]:
models = {
    'Shallow MLP': nn.Sequential(
        nn.Linear(128*128, 64),
        nn.ReLU(),
        nn.Linear(64, 3)
    ),
    'Deep MLP': nn.Sequential(
        nn.Linear(128*128, 256),
        nn.ReLU(),
        nn.Linear(256, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, 3)
    ),
    'Shallow CNN': nn.Sequential(
        nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Flatten(),
        nn.Linear(32*64*64, 3) 
    ),
    'Deep CNN': nn.Sequential(
        nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Flatten(),
        nn.Linear(64*32*32, 128),  
        nn.ReLU(),
        nn.Linear(128, 3)
    )

}

In [5]:
class AIdetec_CNN(Dataset):
    def __init__(self, data_dir="data", img_size=128):
        self.data_dir = data_dir
        self.img_dir = os.path.join(data_dir, "initial_data")
        self.df = pd.read_csv(os.path.join(data_dir, "initial_data.csv"))
        
        self.transform = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor()
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_name = os.path.basename(row["file_name"])  
        label = int(row["label"])

        img_path = os.path.join(self.img_dir, img_name)
        if not os.path.exists(img_path):
            raise FileNotFoundError(f"No se encontró la imagen: {img_path}")

        img = Image.open(img_path).convert("RGB")
        img = self.transform(img)

        return img, label


In [6]:
class AIdetec_MLP(Dataset):
    def __init__(self, data_dir="data", img_size=128):
        self.data_dir = data_dir
        self.img_dir = os.path.join(data_dir, "initial_data")
        self.df = pd.read_csv(os.path.join(data_dir, "initial_data.csv"))
        
        self.transform = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            transforms.Grayscale(),
            transforms.Lambda(lambda x: x.view(-1))
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_name = os.path.basename(row["file_name"])  
        label = int(row["label"])

        img_path = os.path.join(self.img_dir, img_name)
        if not os.path.exists(img_path):
            raise FileNotFoundError(f"No se encontró la imagen: {img_path}")

        img = Image.open(img_path).convert("RGB")
        img = self.transform(img)

        return img, label


In [7]:
dataset_CNN = AIdetec_CNN('../data', 128)

train_size_CNN = 800    
val_size_CNN = 200

train_CNN, val_CNN = random_split(dataset_CNN, [train_size_CNN, val_size_CNN])




dataset_MLP = AIdetec_MLP('../data', 128)

train_size_MLP = 800    
val_size_MLP = 200

train_MLP, val_MLP = random_split(dataset_MLP, [train_size_MLP, val_size_MLP])

In [8]:
dl_train_CNN = DataLoader(train_CNN, batch_size=32, shuffle=True)
dl_val_CNN = DataLoader(val_CNN, batch_size=32, shuffle=True)

dl_train_MLP = DataLoader(train_MLP, batch_size=32, shuffle=True)
dl_val_MLP = DataLoader(val_MLP, batch_size=32, shuffle=True)

In [9]:
size = 0
for batch_x, batch_y in dl_train_MLP:
  size += batch_x.shape[0]
  print(batch_x.shape), print(batch_y.shape)
print("Total size: ", size)

torch.Size([32, 16384])
torch.Size([32])
torch.Size([32, 16384])
torch.Size([32])
torch.Size([32, 16384])
torch.Size([32])
torch.Size([32, 16384])
torch.Size([32])
torch.Size([32, 16384])
torch.Size([32])
torch.Size([32, 16384])
torch.Size([32])
torch.Size([32, 16384])
torch.Size([32])
torch.Size([32, 16384])
torch.Size([32])
torch.Size([32, 16384])
torch.Size([32])
torch.Size([32, 16384])
torch.Size([32])
torch.Size([32, 16384])
torch.Size([32])
torch.Size([32, 16384])
torch.Size([32])
torch.Size([32, 16384])
torch.Size([32])
torch.Size([32, 16384])
torch.Size([32])
torch.Size([32, 16384])
torch.Size([32])
torch.Size([32, 16384])
torch.Size([32])
torch.Size([32, 16384])
torch.Size([32])
torch.Size([32, 16384])
torch.Size([32])
torch.Size([32, 16384])
torch.Size([32])
torch.Size([32, 16384])
torch.Size([32])
torch.Size([32, 16384])
torch.Size([32])
torch.Size([32, 16384])
torch.Size([32])
torch.Size([32, 16384])
torch.Size([32])
torch.Size([32, 16384])
torch.Size([32])
torch.Size([32, 

In [10]:
def train_model(model, train_dl, val_dl, epochs=1000, lr=0.1):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    device = torch.device("cpu")
    model.to(device)

    train_loss_history = []
    train_acc_history = []
    val_loss_history = []
    val_acc_history = []

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0

        for batch_x, batch_y in train_dl:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            optimizer.zero_grad()
            output = model(batch_x)
            loss = criterion(output, batch_y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            train_correct += (predicted == batch_y).sum().item()

        train_loss_history.append(train_loss / len(train_dl))
        train_acc_history.append(train_correct / len(train_dl.dataset))

        model.eval()
        val_loss = 0.0
        val_correct = 0
        with torch.no_grad():
            for batch_x, batch_y in val_dl:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                output = model(batch_x)
                loss = criterion(output, batch_y)
                val_loss += loss.item()
                _, predicted = torch.max(output.data, 1)
                val_correct += (predicted == batch_y).sum().item()

        val_loss_history.append(val_loss / len(val_dl))
        val_acc_history.append(val_correct / len(val_dl.dataset))

        print(f"Epoch {epoch}, Train Loss: {train_loss_history[-1]:.4f}, Train Acc: {train_acc_history[-1]:.4f}, Val Loss: {val_loss_history[-1]:.4f}, Val Acc: {val_acc_history[-1]:.4f}")



    return train_loss_history, train_acc_history, val_loss_history, val_acc_history


In [11]:
train_loss_history, train_acc_history, val_loss_history, val_acc_history = train_model(models['Shallow MLP'], dl_train_MLP, dl_val_MLP, epochs=10, lr=0.001)

Epoch 0, Train Loss: 2.6420, Train Acc: 0.4775, Val Loss: 1.7237, Val Acc: 0.5200
Epoch 1, Train Loss: 0.9926, Train Acc: 0.5288, Val Loss: 0.6890, Val Acc: 0.5400
Epoch 2, Train Loss: 0.8053, Train Acc: 0.5737, Val Loss: 1.1207, Val Acc: 0.4850
Epoch 3, Train Loss: 0.7239, Train Acc: 0.5850, Val Loss: 0.7040, Val Acc: 0.5400
Epoch 4, Train Loss: 0.6776, Train Acc: 0.6038, Val Loss: 0.8943, Val Acc: 0.5200
Epoch 5, Train Loss: 0.6479, Train Acc: 0.6288, Val Loss: 0.7774, Val Acc: 0.5050
Epoch 6, Train Loss: 0.7484, Train Acc: 0.6038, Val Loss: 0.7529, Val Acc: 0.5300
Epoch 7, Train Loss: 0.6631, Train Acc: 0.6200, Val Loss: 0.7683, Val Acc: 0.5600
Epoch 8, Train Loss: 0.6862, Train Acc: 0.6325, Val Loss: 0.8540, Val Acc: 0.4950
Epoch 9, Train Loss: 0.5830, Train Acc: 0.6937, Val Loss: 0.7293, Val Acc: 0.5800


In [12]:
train_loss_history, train_acc_history, val_loss_history, val_acc_history = train_model(models['Deep MLP'], dl_train_MLP, dl_val_MLP, epochs=10, lr=0.001)

Epoch 0, Train Loss: 0.8936, Train Acc: 0.5100, Val Loss: 0.6947, Val Acc: 0.5350
Epoch 1, Train Loss: 0.7228, Train Acc: 0.4800, Val Loss: 0.8479, Val Acc: 0.4800
Epoch 2, Train Loss: 0.7602, Train Acc: 0.5150, Val Loss: 0.6961, Val Acc: 0.5150
Epoch 3, Train Loss: 0.6985, Train Acc: 0.5162, Val Loss: 0.7311, Val Acc: 0.5200
Epoch 4, Train Loss: 0.6854, Train Acc: 0.5437, Val Loss: 0.6900, Val Acc: 0.4650
Epoch 5, Train Loss: 0.6797, Train Acc: 0.5475, Val Loss: 0.7049, Val Acc: 0.5300
Epoch 6, Train Loss: 0.6672, Train Acc: 0.5950, Val Loss: 0.7115, Val Acc: 0.4900
Epoch 7, Train Loss: 0.6994, Train Acc: 0.5387, Val Loss: 0.7342, Val Acc: 0.5300
Epoch 8, Train Loss: 0.6667, Train Acc: 0.5850, Val Loss: 0.7058, Val Acc: 0.5000
Epoch 9, Train Loss: 0.6782, Train Acc: 0.5687, Val Loss: 0.6813, Val Acc: 0.5100


In [13]:
train_loss_history, train_acc_history, val_loss_history, val_acc_history = train_model(models['Shallow CNN'], dl_train_CNN, dl_val_CNN, epochs=10, lr=0.001)

Epoch 0, Train Loss: 3.7366, Train Acc: 0.5825, Val Loss: 0.5393, Val Acc: 0.7900
Epoch 1, Train Loss: 0.5669, Train Acc: 0.7638, Val Loss: 0.6738, Val Acc: 0.7350
Epoch 2, Train Loss: 0.2921, Train Acc: 0.8838, Val Loss: 0.3990, Val Acc: 0.8300
Epoch 3, Train Loss: 0.2547, Train Acc: 0.8962, Val Loss: 0.3265, Val Acc: 0.8550
Epoch 4, Train Loss: 0.1557, Train Acc: 0.9425, Val Loss: 0.3225, Val Acc: 0.8500
Epoch 5, Train Loss: 0.1026, Train Acc: 0.9738, Val Loss: 0.3312, Val Acc: 0.8550
Epoch 6, Train Loss: 0.1008, Train Acc: 0.9762, Val Loss: 0.3831, Val Acc: 0.8700
Epoch 7, Train Loss: 0.0633, Train Acc: 0.9838, Val Loss: 0.3010, Val Acc: 0.8650
Epoch 8, Train Loss: 0.0474, Train Acc: 0.9938, Val Loss: 0.4378, Val Acc: 0.8550
Epoch 9, Train Loss: 0.0437, Train Acc: 0.9925, Val Loss: 0.3740, Val Acc: 0.8750


In [14]:
train_loss_history, train_acc_history, val_loss_history, val_acc_history = train_model(models['Deep CNN'], dl_train_CNN, dl_val_CNN, epochs=10, lr=0.001)

Epoch 0, Train Loss: 0.8053, Train Acc: 0.5675, Val Loss: 0.7279, Val Acc: 0.6500
Epoch 1, Train Loss: 0.4187, Train Acc: 0.8125, Val Loss: 0.4317, Val Acc: 0.8550
Epoch 2, Train Loss: 0.2635, Train Acc: 0.8938, Val Loss: 0.3102, Val Acc: 0.8550
Epoch 3, Train Loss: 0.2354, Train Acc: 0.9062, Val Loss: 0.2988, Val Acc: 0.8450
Epoch 4, Train Loss: 0.2286, Train Acc: 0.9187, Val Loss: 0.5622, Val Acc: 0.8300
Epoch 5, Train Loss: 0.1837, Train Acc: 0.9425, Val Loss: 0.4752, Val Acc: 0.8100
Epoch 6, Train Loss: 0.1207, Train Acc: 0.9637, Val Loss: 0.3181, Val Acc: 0.8600
Epoch 7, Train Loss: 0.0905, Train Acc: 0.9750, Val Loss: 0.3808, Val Acc: 0.8100
Epoch 8, Train Loss: 0.1271, Train Acc: 0.9625, Val Loss: 0.3051, Val Acc: 0.8700
Epoch 9, Train Loss: 0.0468, Train Acc: 0.9850, Val Loss: 0.3814, Val Acc: 0.8600
